## Задача работы - построить скоринг-модель для вторичных клиентов банка, которая бы предсказывала вероятность дефолта клиента

Описания полей

* client_id - идентификатор клиента

* education - уровень образования

* sex - пол заемщика

* age - возраст заемщика

* car - флаг наличия автомобиля

* car_type - флаг автомобиля иномарки

* decline_app_cnt - количество отказанных прошлых заявок

* good_work - флаг наличия “хорошей” работы

* bki_request_cnt - количество запросов в БКИ

* home_address - категоризатор домашнего адреса

* work_address - категоризатор рабочего адреса

* income - доход заемщика

* foreign_passport - наличие загранпаспорта

* sna - связь заемщика с клиентами банка

* first_time - давность наличия информации о заемщике

* score_bki - скоринговый балл по данным из БКИ

* region_rating - рейтинг региона

* app_date - дата подачи заявки

* default - флаг дефолта по кредиту

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Импортируем библиотеки

In [ ]:
import pandas as pd
import numpy as np
from pandas import Series
import random

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

import operator

import calendar

In [ ]:
# Избавляемся от предупреждений
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def diagram_bar(data, column):
    '''
     Функция построения гистограммы и boxplot в одном окне
     
     Входные параметры: data - датафрейм, 
                        column - признак (столбец), по которому строятся графики
    '''
    fig = plt.figure()
    main_axes = fig.add_axes([0,0,1,1])
    data[column].hist(bins = 20)  # Построение гистограммы
    
    insert_axes = fig.add_axes([1.1,0,0.5,1])
    data.boxplot(column = column)  # Построение boxplot

In [ ]:
def model_visual(predict, pro_babs, X_tst, y_tst):
    ''' 
    Функция визуализации результатов работы модели: метрики, confusion matrix, ROC-AUC
    
    Входные параметры:  predict - массив предсказанных значений целевой переменной
                        pro_babs - вероятность принадлежности предсказанных значений целевой переменной к имеющимся классам
                        X_tst - массив тестовых значений признаков
                        y_tst - массив тестовых значений целевой переменной
    '''   
    # Печать отчета о классификации
    print(classification_report(y_tst, predict))
    
    # Печать confusion matrix
    conf_mat = confusion_matrix(predict, y_test)
    print('Confusion matrix:\n{}'.format(conf_mat))
    
    # Визуализация confusion matrix
    class_names = ['not_default', 'default']
    df_cm = pd.DataFrame(conf_mat, index=class_names, columns=class_names)
    plt.figure(figsize = (10,5))
    sns.heatmap(df_cm, annot=True, fmt="d");
    
    # Построение графика ROC-AUC
    pro_babs = pro_babs[:,1]

    fpr, tpr, threshold = roc_curve(y_test, pro_babs)
    roc_auc = roc_auc_score(y_test, pro_babs)

    plt.figure(figsize=(20,10))
    plt.plot([0, 1], label='Baseline', linestyle='--')
    plt.plot(fpr, tpr, label = 'Regression')
    plt.title('Logistic Regression ROC AUC = %0.5f' % roc_auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc = 'lower right')
    plt.show()

In [ ]:
def clear_sign_num(df_, sign):
    '''
     Функция очиски количественных признаков от выбросов
     
     Входные параметры: df_ - датасет
                        sign - признак, по которому делается очистка
    '''
    for i in range(2): 
        sign_dscrb = df_[df_.default == i][sign].describe()
        delta = (sign_dscrb[6] - sign_dscrb[4])*1.5 # Рассчитываем межквартильный порог
        top_board = sign_dscrb[6] + delta # Верхняя граница выбросов
        bot_board = sign_dscrb[4] - delta # Нижняя граница выбросов
        
        # Очищаем по нижней границе
        df_.drop(df_[df_.default == i][(df_[sign] < bot_board)].index, axis= 0, inplace= True) 
    
        # Очищаем по верхней границе     
        df_.drop(df_[df_.default == i][(df_[sign] > top_board)].index, axis= 0, inplace= True) 

In [ ]:
def G_S_CV(model_entr, param_grid, X_trn, y_trn, X_tst, y_tst):
    '''
     Функция подбора гиперпараметров, построения оптимизированной модели логистической регрессии и отображения результатов
     
     Входные данные: model_entr - оптимизируемая модель
                     param_grid - ограничения для параметра регуляризации и гиперпараметры
                     X_trn - массив тренировочных значений признаков
                     y_trn - массив тренировочных значений целевой переменной
                     X_tst - массив тестовых значений признаков
                     y_tst - массив тестовых значений целевой переменной
    
    Возвращает регуляризированную модель с оптимальными параметрами
    '''
    # Подбор параметров
    model_tmp = GridSearchCV(model_entr, param_grid, scoring='f1', n_jobs=-1, cv=5)


    # Обучение модели
    model_tmp.fit(X_trn, y_trn)
    model_exit = model_tmp.best_estimator_  # Запоминание оптимальных параметров

    # Печать параметров
    best_parameters = model_exit.get_params()
    for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))
        
    # Предсказание целевой переменной
    preds = model_exit.predict(X_tst)
    
    # Вероятность принадлежности предсказанных значений целевой переменной к имеющимся классам
    probs = model_exit.predict_proba(X_tst)
    
    # Визуализация результатов
    model_visual(preds, probs, X_tst, y_tst)
    
    return model_exit

In [ ]:
# Фиксируем RANDOM_SEED
RANDOM_SEED = 42

## Загружаем данные

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-scoring/'
df_train = pd.read_csv(DATA_DIR+'/train.csv')
df_test = pd.read_csv(DATA_DIR+'test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

### Проверяем загруженные данные

In [ ]:
# Тренировочный датасет
df_train.sample(5)

In [ ]:
# Тестовый датасет
df_test.sample(5)

In [ ]:
# Файл "sample_submission"
sample_submission

In [ ]:
# Общая информация о тренировочном датасете
df_train.info()

##### В тренировочном датасете имеем 73799 записей (наблюдений) и 18 признаков: 6 строковых и 12 числовых, а также 1 целевая переменная

In [ ]:
# Проверяем наличие пропущенных значений
df_train.isna().sum()

#### В тренировочном датасете имеется 307 пропущенных значений только в одном признаке - "education"

In [ ]:
# Смотрим количество недефолтных (0) и дефолтных (1) заёмщиков
df_train.default.value_counts()

In [ ]:
plt.figure(figsize=(10, 5))
df_train.default.value_counts().plot.barh()

#### Количество "дефолтных" заёмщиков превышает 10% "недефолтных", поэтому можно считать данную выборку сбалансированной

In [ ]:
# Общая информация о тестовом датасете
df_test.info()

#### В тестовом датасете имеем 36349 записей (наблюдений) и 18 признаков: 6 строковых и 12 числовых (как и в тренировочном датасете)

In [ ]:
# Проверяем наличие пропущенных значений
df_test.isna().sum()

#### В тестовом датасете, как и тренировочном, имеются пропущенные значения только в одном признаке - "education"

In [ ]:
# Для корректной обработки признаков объединяем тренировочный и тестовый датасеты в один датасет
df_train['sample'] = 1  # Помечаем где у нас трейн
df_test['sample'] = 0   # Помечаем где у нас тест
df_test['default'] = 0  # В тесте у нас нет значения default, поэтому мы его просто заполняем нулями

df = df_test.append(df_train, sort=False).reset_index(drop=True) # Объединяем

In [ ]:
# Проверяем результат
df.sample(5)

## Обработка и очистка данных

In [ ]:
# Общая информация об объединённом датасете
df.info()

In [ ]:
# Проверяем наличие пропущенных значений
df.isna().sum()

#### Всего имеется 478 пропущенных значений в признаке "education"

In [ ]:
# Проверяем наличие пустых записей
df[df_test == ''].sum()

#### Пустых записей нет

#### В датасете имеется 478 пропущенных значений в признаке "education", что составляет менее 0.5% от общего числа записей. Данные записи, всилу их малого количества, теоретически можно исключить из дальнейшей обработки, но тогда мы потеряем часть записей из тестового набора данных, которые в конечном итоге мы должны предсказать, что не очень хорошо. Тогда заполним пропущенные данные. Предполагаем, что уровень зарплаты сильно зависит от уровня образования, а также от возраста заемщика

In [ ]:
# Смотрим распределение признака "education"
plt.figure(figsize=(10, 5))
df.education.value_counts().plot.barh()

#### Проверяем зависимость уровня образования от возраста и заработной платы

In [ ]:
# Смотрим значимость признаков 'age' и 'income' для уровня образования
df_tmp = df.dropna()
plt.rcParams['figure.figsize'] = (10,5)
imp_num = Series(f_classif(df_tmp[['age', 'income']], df_tmp['education'])[0], index = ['age', 'income'])
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

#### Признак 'income' является более значимым

In [ ]:
# Графики Boxplot "education-income"
plt.figure(figsize=(15, 10))
sns.boxplot(x="education", y="income", data=df, showfliers=False)

In [ ]:
# Графики Boxplot "education-age"
plt.figure(figsize=(15, 10))
sns.boxplot(x="education", y="age", data=df, showfliers=False)

### Резюме: 
#### Пропущенные значения признака "education" будем заполнять, в зависимости от заработной платы (признак "income") по принципу "выше или равно" медианному значению конкретного уровня обучения (при заполнении пропущенных значений признака "education" в зависимости от возраста метрики получаются чуть хуже)

In [ ]:
# Модуль заполнения пропущенных значений признака "education" в зависимости от зарплаты

edu_encome = {}  # Создаём словарь "образование - медиана зарплаты"

# Заполняем словарь медианными значениями уровня зарплаты ("income") в зависимости от уровня образования ("education")
for edu in df.education.value_counts().index:
    edu_encome[edu] = df[df.education == edu].income.median() # Заполнение словаря
    
# Сортируем словарь по убыванию значений зарплаты
ee_sort = sorted(edu_encome.items(),key = operator.itemgetter(1),reverse = True)

# Заполняем пропуски в признаке "education"
for ind in df[df.education.isna()].index:
    for i, j in ee_sort:
        if df.iloc[ind].income >= j: 
            df.loc[ind, 'education'] = i
            break

# Заполняем оставшиеся пропуски значением "UGR"
df.education.fillna('UGR', inplace=True)

In [ ]:
# Проверяем результат
df.isna().sum()

#### В датасете есть признак "app_date" - дата подачи заявки, имеющий тип "object". Преобразуем его в формат "datetime"

In [ ]:
df['app_date'] = pd.to_datetime(df.app_date)

In [ ]:
# Проверяем
df.info()

In [ ]:
# Смотрим диапазон дат
print(df['app_date'].min())
print(df['app_date'].max())

#### Диапазон дат поданных заявок охватывает период за четыре месяца с начала года. 
#### Преобразуем данный признак в число дней, прошедших с начала года (с момента подачи первой заявки)

In [ ]:
# Запомним на всякий случай наши данные
date_app = df['app_date']

start_date = df['app_date'].min()  # Минимальное значение дат
df['app_date'] = df['app_date'].apply(lambda x: (x - start_date).days)  # Преобразование столбца

In [ ]:
# Проверяем результат
df.sample(5)

In [ ]:
# Посмотрим распределение преобразованного признака "app_date"
df['app_date'].describe()

In [ ]:
plt.figure(figsize=(10, 5))
diagram_bar(df, 'app_date')

In [ ]:
sns_plot = sns.distplot(df['app_date'])
fig = sns_plot.get_figure()

#### Очень похоже на равномерное распределение

In [ ]:
# Посмотрим распределение признака "client_id"
sns_plot = sns.distplot(df['client_id'])
fig = sns_plot.get_figure()

In [ ]:
df['client_id'].describe()

#### Признак "client_id" имеет абсолютно равномерное распределение, по сути является порядковым номером клиента и в состав признаков для построения модели включаться не будет

#### Распределим наши данные на цифровые, категориальные и бинарные переменные

In [ ]:
df.nunique()

In [ ]:
# Таким образом, для построения модели мы берём 17 признаков:

# К цифровым (числовым) относится 6 признаков:
num_cols = ['app_date', 'age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']

# К категориальным относится 6 признаков:
cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']

# К бинарным относится 5 признаков:
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

### Работа с числовыми призаками

In [ ]:
# Посмотрим распределение цифровых признаков
for i in df[num_cols]:
    plt.figure(figsize=(10,5))
    sns.distplot(df[i], kde = False, color='r')
    plt.title(i)
    plt.show()

#### У четырёх из шести признаков:  "age", "income", "decline_app_cnt" и "bki_request_cnt" имеются не очень хорошие "правые хвосты". Логарифмирование данных признаков может поправить положение только у двух из них: "age" и "income". Для признаков "decline_app_cnt" и "bki_request_cnt" логарифмирование вряд ли поможет, так как у них имеется слишком много нулевых значений

In [ ]:
df['decline_app_cnt'].value_counts().head()

In [ ]:
df['bki_request_cnt'].value_counts().head()

In [ ]:
# Преобразуем все числовые переменные в логарифмические функции
for i in num_cols:
    df[i] = np.log1p(df[i].abs())
    plt.figure(figsize=(10,5))
    sns.distplot(df[i][df[i] > 0].dropna(), kde = False, rug=False, color='b')
    plt.title(i)
    plt.show()

#### В распределении признаков "age" и "score_bki" произошло небольшое смещение вправо, распределение признака "income" стало нормальным

#### Посмотрим наличие выбросов у числовых признаков

In [ ]:
# Визуализация числовых переменных с использованием графика boxplot
fig, axes = plt.subplots(2, 3, figsize=(20,15))
plt.subplots_adjust(wspace = 0.5)
axes = axes.flatten()
for i in range(len(num_cols)):
    sns.boxplot(x="default", y=num_cols[i], data=df, orient = 'v', ax=axes[i], showfliers=True)

### Из полученных графиков можно сделать следующие выводы:
#### Пять из шести числовых параметров имеют множественные выбросы
#### Дефолтные клиенты в среднем немного младше недефолтных
#### Дефолтные клиенты в среднем имеют немногим более низкий доход
#### Дефолтные клиенты в среднем имеют бОльшее количество отмененных заявок
#### Дефолтные клиенты в среднем имеют больше запросов в БКИ
#### Дефолтные клиенты в среднем имеют более низкий в абсолютной величине скоринговый балл

In [ ]:
# Оценим корреляцию Пирсона для непрерывных переменных
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(df[num_cols].corr().abs(), vmin=0, vmax=1, annot = True)

### Выводы:
#### Числовые признаки слабо коррелированы друг с другом. Наибольшая корреляция наблюдается между признаками "score_bki" (скоринговый балл), "decline_app_cnt" (количество отказов) и "bki_request_cnt" (количество обращений в БКК). Все числовые признаки оставляем для дальнейшей работы

#### Оценим значимость числовых переменных для целевого признака "default" с помощью функции f_classif. В качестве меры значимости используем значение f-статистики

In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
imp_num = Series(f_classif(df[num_cols], df['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

### Выводы:
#### Из числовых признаков наибольшее влияние на целевую переменную оказывают признаки "score_bki" (скоринговый балл) и "decline_app_cnt" (количество отказов), наименьшее - "age" (возраст) и "app_date" (дата подачи заявки)

### Работа с бинарными признаками

In [ ]:
# Для бинарных признаков используем LabelEncoder
map_categories = {}  # Создаём словарь кодировок

label_encoder = LabelEncoder()

for column in bin_cols:  # Организуем цикл по бинарным признакам и формируем словарь
    df[column] = label_encoder.fit_transform(df[column])
    map_categories[column] = dict(enumerate(label_encoder.classes_)) # Запоминаем, что закодировали

In [ ]:
# Проверяем словарь
map_categories

In [ ]:
# Убеждаемся в преобразовании    
df.head()

### Работа с категориальными признаками

In [ ]:
# Для двух категориальных признаков: "education" и "region_rating" первоначальнор используем LabelEncoder
for name in ['education', 'region_rating']:
    df[name] = label_encoder.fit_transform(df[name])
    map_categories[name] = dict(enumerate(label_encoder.classes_)) # Запоминаем, что закодировали

In [ ]:
# Проверяем словарь
map_categories

#### Для оценки значимости категориальных и бинарных переменных используем функцию mutual_info_classif

In [ ]:
imp_cat = Series(mutual_info_classif(df[bin_cols + cat_cols], df['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

### Выводы:
#### Из категориальных признаков самыми значимыми являются признаки "sna" (связь заемщика с клиентами банка) и "first_time" (давность наличия информации о заемщике), наименее значимыми - "sex" (пол заёмщика)

### Посмотрим зависимости между признаками

In [ ]:
cat_bin_cols = cat_cols + bin_cols

In [ ]:
# Зарплата - категорийные переменные
fig, axes = plt.subplots(4, 3, figsize=(20,25))
plt.subplots_adjust(wspace = 0.5)
axes = axes.flatten()
for i in range(len(cat_bin_cols)):
    sns.boxplot(x=cat_bin_cols[i], y="income", data=df, orient = 'v', ax=axes[i], showfliers=False)

In [ ]:
# Скоринговый бал - категорийные переменные
fig, axes = plt.subplots(4, 3, figsize=(20,25))
plt.subplots_adjust(wspace = 0.5)
axes = axes.flatten()
for i in range(len(cat_bin_cols)):
    sns.boxplot(x=cat_bin_cols[i], y="score_bki", data=df, orient = 'v', ax=axes[i], showfliers=False)

In [ ]:
# Ввозраст - категорийные переменные
fig, axes = plt.subplots(4, 3, figsize=(20,25))
plt.subplots_adjust(wspace = 0.5)
axes = axes.flatten()
for i in range(len(cat_bin_cols)):
    sns.boxplot(x=cat_bin_cols[i], y="age", data=df, orient = 'v', ax=axes[i], showfliers=False)

### Выводы:
#### 1. Уровень запрплаты в среднем не зависит от уровня образования
#### 2. Чем выше рейтинг региона, тем в среднем выше уровень зарплаты
#### 3. Уровень зарплаты практически не зависит от адреса проживания и работы
#### 4. Чем выше уровень зарплаты, тем ниже индекс связи заемщика с клиентами банка
#### 5. Чем выше уровень зарплаты, тем выше признак давности наличия информации о заемщике
#### 5. Клиенты мужского пола в среднем получают более высокую зарплату
#### 6. Клиенты с болеее высокой зарплатой чаще имеют машину, у них есть хорошая работа, они чаще выезжают за границу (на отдых)
#### 7. Чем выше индекс образования, тем, в-среднем, ниже скоринговый балл
#### 8. Скоринговый балл не зависит от адреса проживания и работы, рейтинга региона и индекса связи заемщика с клиентами банка
#### 9. Чем выше значение индекса давности наличия информации о заемщике, тем, в-среднем, выше скоринговый балл
#### 10. Более молодые клиенты предпочитают проживать в регионах с более высоким рейтингом
#### 11. Среди клиентов мужчины раньше начинают обращаться в банки, также мужчины раньше покупают машину, получают хорошую работу и оформляют загранпаспорт

In [ ]:
df_copy = df.copy()  # Запоминаем на всякий случай датасет до "get_dummies"

# "Оцифровываем" категориальные переменные
for sign in cat_cols:
    df = pd.get_dummies(df, columns=[sign])

In [ ]:
# Проверяем результат  
df.head()

In [ ]:
# Оценим корреляцию Пирсона для всех признаков преобразованного датасета
plt.rcParams['figure.figsize'] = (45,30)
sns.heatmap(df.corr().abs(), vmin=0, vmax=1, annot = True)

### Выводы:
#### Высокая корреляция (>0.7) наблюдается между следующими признаками:
#### 1. "client_id" и "app_date" = 1.0. Исключим из модели признак "client_id"
#### 2. "home_address_1" и "home_address_2" = 0.97
#### 3. "home_address_1" и "work_address_3" = 0.82
#### 4. "home_address_2" и "work_address_3" = 0.80
#### 5. "work_address_2" и "work_address_3" = 0.78
#### 6. "education_1" и "education_3" = 0.72
#### 7. "car" и "car_type" = 0.70
#### Посмотрим влияние данных признаков на поведение модели

In [ ]:
# Смотрим общую информацию по преобразованному датасету
df.info()

In [ ]:
df.shape

## Подготовка данных к машинному обучению

In [ ]:
# Разбиваем датасет на тренировочный и тестовый, удалив лишние столбцы
train_data = df.query('sample == 1').drop(['sample', 'client_id'], axis=1)  # Тренировочный
test_data = df.query('sample == 0').drop(['sample', 'default'], axis=1)     # Тестовый

# Сохраняем ID клиентов из тестового набора для  формирования Submission
id_test = test_data['client_id']

# Удаляем ID клиентов из тестового набора для последующего формирования признакового проостранства
test_data.drop(['client_id'], axis=1, inplace = True)

In [ ]:
print(train_data.shape, test_data.shape)

### Стандартизировать числовые переменные будем методом "StandardScaler", так как метод "RobustScaler" показал несколько худшие результаты

In [ ]:
# Тренировочный датасет
X_num_train = StandardScaler().fit_transform(train_data[num_cols].values)
#X_num_train = RobustScaler().fit_transform(train_data[num_cols].values)

# Тестовый датасет
X_num_test = StandardScaler().fit_transform(test_data[num_cols].values)
#X_num_test = RobustScaler().fit_transform(test_data[num_cols].values)

In [ ]:
# Формируем датасеты с категориальными переменными

# Тренировочный датасет
df_cat_train = train_data.drop(["default"], axis = 1)
df_cat_train.drop(num_cols, axis = 1, inplace = True)

# Тестовый датасет
df_cat_test = test_data.drop(num_cols, axis = 1)

In [ ]:
# Объединяем цифровые, бинарные и категориальные оцифрованные признаки в одну матрицу (в одно признаковое пространство)

# Тренировочные данные
X_1 = np.hstack([X_num_train, df_cat_train.values])

# Тестовые данные
X_t_1 = np.hstack([X_num_test, df_cat_test.values])

In [ ]:
# Целевая переменная
y_1 = train_data['default'].values

In [ ]:
print(X_1.shape, X_t_1.shape, y_1.shape)

## МОДЕЛЬ № 1 (базовая)

### Первую (базовую) модель построим по всем признакам без фильтрации

### Обучение модели логистической регрессии

In [ ]:
# Разбиваем данные на тренировочную и тестовую части
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size=0.20, shuffle = True, random_state=RANDOM_SEED)

In [ ]:
# Строим и обучаем модель
model_1 = LogisticRegression(multi_class = 'ovr', class_weight='balanced', solver='liblinear', random_state=RANDOM_SEED)
model_1.fit(X_train, y_train)

In [ ]:
# Предсказываем целевую переменную и вероятностные оценки
y_pred = model_1.predict(X_test)
probs = model_1.predict_proba(X_test)

#### В качестве метода оценки прогностической способности модели используем ROC-анализ

In [ ]:
# Визуализация результатов
model_visual(y_pred, probs, X_test, y_test)

## ВЫВОДЫ (модель №1):
### Дефолтные клиенты угадываются достаточно неплохо: 1245 из 1827 (68%), при этом угадывание недефолтных клиентов составляет 67% (8692 из 12933). Значение ROC AUC достаточно высокое - 0.74328.
### Данная модель примерно одинаково предсказывает дефолтных и недефолтных клиентов

## МОДЕЛЬ № 1.1 (базовая + регуляризация)[](http://)

### Подбор гиперпараметров для модели

In [ ]:
# Ограничения для параметра регуляризации 
#C = np.logspace(0, 4, 5)
C = [0.1, 1, 10]

# ... и гиперпараметры
iter_ = 150
epsilon_stop = 1e-4

param_grid_1 = [
    {'penalty': ['l1'], 'C': C, 'max_iter':[iter_],'tol':[epsilon_stop]},
    {'penalty': ['l2'], 'C': C, 'max_iter':[iter_],'tol':[epsilon_stop]},
    {'penalty': ['none'], 'max_iter':[iter_],'tol':[epsilon_stop]},
               ]

param_grid_2 = [
    {'penalty': ['l1'],
     'C': C,
     'solver': ['liblinear', 'lbfgs'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['l2'], 
     'C': C,
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['none'], 
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
              ]

In [ ]:
# Регуляризация построенной модели с подбором гиперпараметров param_grid_1
model_1_1 = G_S_CV(model_1, param_grid_1, X_train, y_train, X_test, y_test)

In [ ]:
# Регуляризация построенной модели с подбором гиперпараметров param_grid_2
#model_1_2 = G_S_CV(model_1, param_grid_2, X_train, y_train, X_test, y_test)

### ВЫВОДЫ (модель № 1.1):
### Значения всех метрик остались без изменения

## Модель № 2 (подбор оптимального состава признаков)

In [ ]:
# Исключаем из построения модели из состава числовых данных малозначимые признаки "app_date" и "age"
num_cols_no_date = ['age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']
num_cols_no_age = ['app_date', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']
num_cols_no_da = ['decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']

#### Исключение из модели любого числового признака приводит к ухудшению её предсказательной способности. По этой причине числовые признаки оставляем в модели все

In [ ]:
# Стандартизируем числовые переменные 
#X_num_train = StandardScaler().fit_transform(train_data[num_cols].values)  # Тренировочный датасет
#X_num_train = RobustScaler().fit_transform(train_data[num_cols].values)

#X_num_test = StandardScaler().fit_transform(test_data[num_cols].values)    # Тестовый датасет
#X_num_train = RobustScaler().fit_transform(train_data[num_cols].values)

#### Проверка различных гипотез по составу категориальных признаков в модели показал, что лучшие результаты показала модель без признаков "work_address_3" и "home_address_1", имеющих высокую корреляцию (>0.7) друг с другом и другими переменными. Поэтому, исключаем их из модели

In [ ]:
# Формируем датасеты с категориальными переменными, исключив из их состава признаки,
# имеющую высокую корреляцию друг с другом: "work_address_3" и "home_address_1"

# Тренировочный датасет
df_cat_train = train_data.drop(num_cols, axis = 1)
df_cat_train.drop(["work_address_3", "home_address_1", 'education_1', "default"], axis = 1, inplace = True)
 
# Тестовый датасет
df_cat_test = test_data.drop(num_cols, axis = 1)
df_cat_test.drop(["work_address_3", "home_address_1", 'education_1'], axis = 1, inplace = True)


# Объединяем цифровые, бинарные и категориальные оцифрованные признаки в одну матрицу (в одно признаковое пространство)
X_2 = np.hstack([X_num_train, df_cat_train.values])  # Тренировочные данные
X_t_2 = np.hstack([X_num_test, df_cat_test.values])  # Тестовые данные


# Целевая переменная
y_2 = train_data['default'].values

In [ ]:
# Разбиваем данные на тренировочную и тестовую части
X_train, X_test, y_train, y_test = train_test_split(X_2, y_2, test_size=0.20, shuffle = True, random_state=RANDOM_SEED)


# Строим и обучаем модель
model_2 = LogisticRegression(multi_class = 'ovr', class_weight='balanced', solver='liblinear', random_state=RANDOM_SEED)
model_2.fit(X_train, y_train)

In [ ]:
# Предсказываем целевую переменную и вероятностные оценки
y_pred = model_2.predict(X_test)
probs = model_2.predict_proba(X_test)

# Визуализация результатов
model_visual(y_pred, probs, X_test, y_test)

In [ ]:
# Регуляризация построенной модели с подбором гиперпараметров из param_grid_1
model_2_1 = G_S_CV(model_2, param_grid_1, X_train, y_train, X_test, y_test)

In [ ]:
# Регуляризация построенной модели с подбором гиперпараметров из param_grid_2
#model_2_2 = G_S_CV(model_2_tmp, param_grid_2, X_train, y_train, X_test, y_test)

### ВЫВОДЫ (модели №№ 2 и 2.1):
### Значения метрик относительно модели №1 не ухудшилмсь. Соотношение угадываемых дефолтных и недефолтных клиентов осталось таким же, как и в предыдущих моделях: 68% к 67%. Число угадываемых дефолтных клиентов такое же, как и в модели №1 - 1245. Значение ROC AUC осталось без изменений

## Модель №3 (с фильтрацией признаков и очисткой от выбросов)

In [ ]:
df_clear = train_data.copy()

### При построении базовой модели (model_1) графики boxplot показали, что пять из шести признаков имеют множественные выбросы. Проведём очистку от выбросов  четырёх из них (очистка признака "decline_app_cnt" приводит к удалению слишком большого количества данных, после чего модель показывает не очень хорошие результаты)

In [ ]:
# Очистка числовых признаков от выбросов
column = ['app_date', 'score_bki', 'bki_request_cnt', 'income']
for i in column:
    clear_sign_num(df_clear, i)

In [ ]:
# Посмотрим, на сколько записей уменьшился наш датасет
print(len(train_data), len(df_clear), len(train_data) - len(df_clear))

#### Количество записей уменьшилось на 4831 (6.5%)

In [ ]:
# Проверяем результат
fig, axes = plt.subplots(2, 3, figsize=(20,15))
plt.subplots_adjust(wspace = 0.5)
axes = axes.flatten()
for i in range(len(num_cols)):
    sns.boxplot(x="default", y=num_cols[i], data=df_clear, orient = 'v', ax=axes[i], showfliers=True)

#### Количество выбросов значительно уменьшилось

In [ ]:
# Стандартизируем числовые переменные методом StandardScaler

# Тренировочный датасет
X_num_train = StandardScaler().fit_transform(df_clear[num_cols].values)
#X_num_train = RobustScaler().fit_transform(df_clear[num_cols].values)

# Тестовый датасет
#X_num_test = StandardScaler().fit_transform(test_data[num_cols].values)
#X_num_test = RobustScaler().fit_transform(test_data[num_cols].values)

In [ ]:
# Формируем датасеты с категориальными переменными

# Тренировочный датасет
df_cat_train = df_clear.drop(num_cols, axis = 1)
#df_cat_train.drop(["default"], axis = 1, inplace = True)
df_cat_train.drop(["work_address_3", "home_address_1", 'education_1', "default"], axis = 1, inplace = True)

# Тестовый датасет
df_cat_test = test_data.drop(num_cols, axis = 1)
df_cat_test.drop(["work_address_3", "home_address_1", 'education_1'], axis = 1, inplace = True)


In [ ]:
# Объединяем цифровые, бинарные и категориальные оцифрованные признаки в одну матрицу (в одно признаковое пространство)

# Тренировочные данные
X_3 = np.hstack([X_num_train, df_cat_train.values])

# Тестовые данные
X_t_3 = np.hstack([X_num_test, df_cat_test.values])

# Целевая переменная
y_3 = df_clear['default'].values

In [ ]:
# Разбиваем данные на тренировочную и тестовую части в соотношении 75% на 25%
X_train, X_test, y_train, y_test = train_test_split(X_3, y_3, test_size=0.25, shuffle = True, random_state=RANDOM_SEED)

model_3 = LogisticRegression(multi_class = 'ovr', class_weight='balanced', solver='liblinear', random_state=RANDOM_SEED)
model_3.fit(X_train, y_train)

# Предсказываем целевую переменную и вероятностные оценки
y_pred = model_3.predict(X_test)
probs = model_3.predict_proba(X_test)

# Визуализация результатов
model_visual(y_pred, probs, X_test, y_test)

In [ ]:
# Регуляризация построенной модели с подбором гиперпараметров из param_grid_1
model_3_1 = G_S_CV(model_3, param_grid_1, X_train, y_train, X_test, y_test)

In [ ]:
# Регуляризация построенной модели с подбором гиперпараметров из param_grid_2
#model_3_2 = G_S_CV(model_3, param_grid_2, X_train, y_train, X_test, y_test)

### ВЫВОДЫ (модели №№ 3 и 3.1):
### После проведённой очистки от выбросов четырёх числовых признаков размер датасета уменьшился на 6.5%. При этом, соотношение угадываемых дефолтных и недефолтных клиентов улучшилось и стало равным по 68%. Значение ROC AUC также улучшилось и стало равно 0.74538, причём, регуляризация модели несколько ухудшило значения метрик

## Модель №4 (новые признаки)

#### Добавим в датасет новые признаки: уровень благосостояния ("wealthe", 0-3, зависит от зарплаты), уровень благополучия ("welfar", сумма признаков "wealthe", "car", "car_type", "good_work", "region_rating"), уровень благонадёжности ("reliability", сумма "sna" и "first_time", умноженные на стандартизированные значения "income" и "score_bki"), активность в банковском секторе ("activity", сумма числа запросов в БКИ и числа отказов)

In [ ]:
# Делаем копию нашего датасета после предобработки
df_dop = df_copy.copy()

In [ ]:
# Новый бинарный признак: запросы в БКИ (0-не было, 1-были)
df_dop['bki_req'] = df_dop['bki_request_cnt'].apply(lambda x: 1 if x > 0 else 0)

# Новый бинарный признак: отказы (0-не было, 1-были)
df_dop['dec_app'] = df_dop['decline_app_cnt'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
# Новый бинарный признак: активность в банковском секторе
df_dop['activity'] = df_dop['decline_app_cnt'] + df_dop['bki_request_cnt']
sign_dscrb = df_dop['activity'].describe()
df_dop['activity'] = df_dop['activity'].apply(lambda x: 1 if x >= sign_dscrb[5] else 0)

#  Смотрим результат
df_dop['activity'].value_counts()

In [ ]:
# Новый категориальный признак: уровень благосостояния 
sign_dscrb = df_dop['income'].describe()

# "Оцифровываем" признак "income": 75%<="2" (высокий), 25%<="1"<50% (средний), "0"<25% (низкий)
df_dop['wealthe'] = df_dop['income'].apply(lambda x: 2 if x >= sign_dscrb[6] else 1 if x >= sign_dscrb[4] else 0)

#  Смотрим результат
df_dop['wealthe'].value_counts()

In [ ]:
# Новый признак: уровень благополучия
df_dop['welfare'] = df_dop['car'] + df_dop['car_type'] + df_dop['good_work'] + df_dop['wealthe'] + df_dop['region_rating']

#  Смотрим результат
df_dop['welfare'].value_counts()

In [ ]:
# Попробуем добавить новый категориальный признак - день недели подачи заявки
tmp = pd.DataFrame(date_app)
df_dop['week_day'] = tmp['app_date'].apply(lambda x: calendar.day_abbr[x.date().weekday()])

In [ ]:
# Преобразуем новый признак в числовой вид

# Для нового признака "week_day" первоначально используем LabelEncoder
df_dop['week_day'] = label_encoder.fit_transform(df_dop['week_day'])

# Запоминаем, что закодировали
map_categories['week_day'] = dict(enumerate(label_encoder.classes_))

In [ ]:
# Проверяем
map_categories

In [ ]:
# Добавляем в список с категориальными признаками новые признаки 'wealthe' и 'welfare'
cat_cols_new = cat_cols + ['week_day', 'wealthe', 'welfare']
cat_cols_new

In [ ]:
# Добавляем в список с бинарными признаками новый признак "activity", "bki_req", "dec_app"
bin_cols_new = bin_cols + ['activity', 'bki_req', 'dec_app']
bin_cols_new

In [ ]:
# Посмотрим значимость новых категориальных признаков для целевой переменной
plt.rcParams['figure.figsize'] = (10,10)
imp_num = Series(f_classif(df_dop[cat_cols_new + bin_cols_new], df_dop['default'])[0], index = cat_cols_new + bin_cols_new)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

#### Новый категориальный признак "week_day" оказался ничтожно значимым для целевой переменной, поэтому в состав признаков для построения модели мы его включать не будем. Другие новые признаки оказались достаточно значимыми, их оставим для построения модели

In [ ]:
# Удаляем из датасета признак "week_day"
df_dop.drop(['week_day'], axis = 1, inplace = True)

# Корректируем список с категорийными переменными
cat_cols_new.remove('week_day')

cat_cols_new

### Новые числовые признаки

In [ ]:
# Нормализация признаков "income", "score_bki", "decline_app_cnt", "bki_request_cnt"
for sign in ['income', 'score_bki', 'decline_app_cnt', 'bki_request_cnt']:
    sign_n = sign + '_norm'
    dscrb = df_dop[sign].describe()
    df_dop[sign_n] = df_dop[sign].apply(lambda x: (x-dscrb[3]) / (dscrb[7] - dscrb[3]))

In [ ]:
# Новый признак: уровень надёжности
df_dop['reliability'] = df_dop['sna']/4 * df_dop['first_time']/4 * (1+df_dop['score_bki_norm']) * (1+df_dop['income_norm'])
                         # * (1 - df_dop['decline_app_cnt_norm'])

In [ ]:
# Удаляем временные признаки 'income_norm', 'score_bki_norm', 'decline_app_cnt_norm'
df_dop.drop(['income_norm', 'score_bki_norm', 'decline_app_cnt_norm', 'bki_request_cnt_norm'], axis = 1, inplace = True)

In [ ]:
# Создаём новый список с числовыми признаками
num_cols_new = num_cols + ['reliability']
num_cols_new

In [ ]:
# Посмотрим значимость новых числовых признаков для целевой переменной
plt.rcParams['figure.figsize'] = (10,5)
imp_num = Series(f_classif(df_dop[num_cols_new], df_dop['default'])[0], index = num_cols_new)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

#### Новый числовые признаки "reliability" и "activity" оказались достаточно значимыми, их добавим в состав признаков для построения модели

#### Таким образом, в наш датасет добавилось 4 новых признака: два категорийных ("wealthe" и "welfare") и два числовых ("reliability" и "activity"). Признак "activity" является обобщающими признаком двух других: "decline_app_cnt" и "bki_request_cnt"

In [ ]:
# "Оцифровываем" категориальные переменные
for sign in cat_cols_new:
    df_dop = pd.get_dummies(df_dop, columns=[sign])

In [ ]:
# Оценим корреляцию Пирсона для всех признаков преобразованного датасета
plt.rcParams['figure.figsize'] = (45,30)
sns.heatmap(df_dop.corr().abs(), vmin=0, vmax=1, annot = True)

In [ ]:
# Удаляем из датасета категорийные признаки с корреляцией >0.7
df_dop.drop(['education_3', 'work_address_3', 'home_address_1', 'wealthe_2'], axis = 1, inplace = True)

In [ ]:
# Корректируем список с числовыми признаками
num_cols_cor = num_cols_new.copy()
num_cols_cor.remove('bki_request_cnt')  # Убираем признак "bki_request_cnt"
num_cols_cor.remove('decline_app_cnt')  # Убираем признак "decline_app_cnt"
num_cols_cor.remove('income')  # Убираем признак "income", так вместо него мы ввели категориальный признак "wealthe"
num_cols_cor

In [ ]:
# Разбиваем датасет на тренировочный и тестовый, удалив лишние столбцы
train_data_dop = df_dop.query('sample == 1').drop(['sample', 'client_id'], axis=1)  # Тренировочный
test_data_dop = df_dop.query('sample == 0').drop(['sample', 'default'], axis=1)     # Тестовый

# Удаляем ID клиентов из тестового набора для последующего формирования признакового проостранства
test_data_dop.drop(['client_id'], axis=1, inplace = True)

In [ ]:
# Стандартизируем числовые данные

# Тренировочный датасет
X_num_train = StandardScaler().fit_transform(train_data_dop[num_cols_cor].values)
#X_num_train = RobustScaler().fit_transform(train_data_dop[num_cols_cor].values)

# Тестовый датасет
X_num_test = StandardScaler().fit_transform(test_data_dop[num_cols_cor].values)
#X_num_test = RobustScaler().fit_transform(test_data_dop[num_cols_cor].values)

In [ ]:
# Формируем датасеты с категориальными переменными

# Тренировочный датасет
df_cat_train = train_data_dop.drop(['default'], axis = 1)
df_cat_train.drop(num_cols_new, axis = 1, inplace = True)

# Тестовый датасет
df_cat_test = test_data_dop.drop(num_cols_new, axis = 1)


# Объединяем цифровые, бинарные и категориальные оцифрованные признаки в одну матрицу (в одно признаковое пространство)
X_4 = np.hstack([X_num_train, df_cat_train.values])
X_t_4 = np.hstack([X_num_test, df_cat_test.values])

# Целевая переменная
y_4 = train_data_dop['default'].values

In [ ]:
# Разбиваем данные на тренировочную и тестовую части
X_train, X_test, y_train, y_test = train_test_split(X_4, y_4, test_size=0.2, shuffle = True, random_state=RANDOM_SEED)

model_4 = LogisticRegression(multi_class = 'ovr', class_weight='balanced', solver='liblinear', random_state=RANDOM_SEED)
model_4.fit(X_train, y_train)

y_pred = model_4.predict(X_test)
probs = model_4.predict_proba(X_test)

# Визуализация результатов
model_visual(y_pred, probs, X_test, y_test)

In [ ]:
# Регуляризация построенной модели
model_4_1 = G_S_CV(model_4, param_grid_1, X_train, y_train, X_test, y_test)

In [ ]:
#model_4_2 = G_S_CV(model_4, param_grid_2, X_train, y_train, X_test, y_test)

### ВЫВОДЫ (модели №№ 4 и 4.1):
### После добавления новых признаков, значения метрик стали лучше, чем у базовой модели (модель №1), и немного хуже, чем в модели №3 

## Модель №5 (модель с новыми признаками (модель №4) + очистка от выбросов)

In [ ]:
df_clear = train_data_dop.copy()

In [ ]:
# Проводим очистку числовых признаков за исключением признака "decline_app_cnt"
#column = ['app_date', 'score_bki', 'income', 'reliability']
column = ['app_date', 'score_bki', 'reliability']
for i in column:
    clear_sign_num(df_clear, i)

# Стандартизируем числовые переменные методом StandardScaler

# Тренировочный датасет
X_num_train = StandardScaler().fit_transform(df_clear[num_cols_cor].values)
#X_num_train = RobustScaler().fit_transform(df_clear[num_cols_cor].values)

# Тестовый датасет
X_num_test = StandardScaler().fit_transform(test_data_dop[num_cols_cor].values)
#X_num_test = RobustScaler().fit_transform(test_data_dop[num_cols_cor].values)


# Формируем датасеты с категориальными переменными

# Тренировочный датасет
df_cat_train = df_clear.drop(num_cols_new, axis = 1)
df_cat_train.drop(["default"], axis = 1, inplace = True)

# Тестовый датасет
df_cat_test = test_data_dop.drop(num_cols_new, axis = 1)


# Объединяем цифровые, бинарные и категориальные оцифрованные признаки в одну матрицу (в одно признаковое пространство)

# Тренировочные данные
X_5 = np.hstack([X_num_train, df_cat_train.values])

# Тестовые данные
X_t_5 = np.hstack([X_num_test, df_cat_test.values])

# Целевая переменная
y_5 = df_clear['default'].values

In [ ]:
# Разбиваем данные на тренировочную и тестовую части
X_train, X_test, y_train, y_test = train_test_split(X_5, y_5, test_size=0.2, shuffle = True, random_state=RANDOM_SEED)

model_5 = LogisticRegression(multi_class = 'ovr', class_weight='balanced', solver='liblinear', random_state=RANDOM_SEED)
model_5.fit(X_train, y_train)

# Предсказываем целевую переменную и вероятностные оценки
y_pred = model_5.predict(X_test)
probs = model_5.predict_proba(X_test)

# Визуализация результатов
model_visual(y_pred, probs, X_test, y_test)

In [ ]:
# Регуляризация построенной модели
model_5_1 = G_S_CV(model_5, param_grid_1, X_train, y_train, X_test, y_test)

### ВЫВОДЫ (модели №№ 5 и 5.1):
### После включения в состав модели новых признаков и очистки от выбросов значение ROC AUC стало равным 0.74891 (максимальным из всех моделей)

## Таким образом, модель 5.1 является самой лучшей моделью по значениям метрик, однако, на соревновании показывает плохие результаты. Очвидно, после очистки признаков от выбросов, происходит переобучение модели. Поэтому, для соревнования (создания файла SUBMISSION) целесообразно использовать модель 4.1.

# SUBMISSION

In [ ]:
predict_submission = model_4_1.predict_proba(X_t_4)[:,1]

In [ ]:
submission = pd.DataFrame({'client_id': id_test, 
                            'default': predict_submission})

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index = False)

# ПОДВАЛ